In [ ]:
from google.colab import drive
drive.mount('/content/drive',force_remount=True)

Mounted at /content/drive


In [ ]:
!pip install -U polars
!pip install kaggle --upgrade
!pip install -U timm

In [ ]:
!mkdir ~/.kaggle
!cp /content/drive/MyDrive/kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json
!kaggle competitions download -c hms-harmful-brain-activity-classification
!sudo apt install -y parallel
!zipinfo -1 /content/hms-harmful-brain-activity-classification.zip | parallel --progress "unzip -q /content/hms-harmful-brain-activity-classification.zip {}"

100% 18.4G/18.4G [14:30<00:00, 21.0MB/s]
100% 18.4G/18.4G [14:30<00:00, 22.7MB/s]
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following additional packages will be installed:
  sysstat
Suggested packages:
  ash csh fish ksh tcsh zsh isag
The following NEW packages will be installed:
  parallel sysstat
0 upgraded, 2 newly installed, 0 to remove and 35 not upgraded.
Need to get 2,434 kB of archives.
After this operation, 4,521 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy-updates/main amd64 sysstat amd64 12.5.2-2ubuntu0.2 [487 kB]
Get:2 http://archive.ubuntu.com/ubuntu jammy/universe amd64 parallel all 20210822+ds-2 [1,947 kB]
Fetched 2,434 kB in 2s (1,216 kB/s)
debconf: unable to initialize frontend: Dialog
debconf: (No usable dialog-like program is installed, so the dialog based frontend cannot be used. at /usr/share/perl5/Debconf/FrontEnd/Dialog.pm line 78, <> line 2.)
debconf: falling ba

In [ ]:
import os
import pickle
import random
import numpy as np
import pandas as pd
import polars as pl
from pathlib import Path
from tqdm.notebook import tqdm
from sklearn.model_selection import StratifiedGroupKFold

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader,Subset
from torch.cuda import amp
import timm

In [ ]:
import sys
sys.path.append('/content/drive/MyDrive/HMS/')

In [ ]:
from src.models.feature_extractor.cnn import CNNSpectrogram
from src.models.feature_extractor.cnn_lstm import CNN_LSTMSpectrogram
from src.models.feature_extractor.secnn import SECNNSpectrogram
from src.models.feature_extractor.spectrogram import SpecFeatureExtractor
from src.models.feature_extractor.panns import PANNsFeatureExtractor
from src.models.feature_extractor.wave import Wavenet_CNNSpectrogram
from src.models.wavenet import wavenet_classifier
from src.models.spec_model import HMSHBACSpecModel

In [ ]:
from src.KL_loss import KLDivLossWithLogits
from src.KL_loss import KLDivLossWithLogitsForVal
from src.HMS_EEG_DATASET import HMS_EEG_Dataset
from src.one_epoch import train_one_epoch
from src.one_epoch import valid_one_epoch

In [ ]:
from cfg.cfg4 import cfg
SUB_NO = 0

In [ ]:
print(vars(cfg))

{'__module__': 'cfg.cfg4', 'EXP_NO': '4', 'model_name': 'seresnext26d_32x4d', 'seed': 48, 'folds': 5, 'epoch': 20, 'batch_size': 50, 'lr': 0.001, 'enable_amp': True, 'device': 'cuda', 'duration': 2000, 'weight_decay': 0.01, 'early_stop': 10, 'kernel_sizes': (32, 16, 2), '__dict__': <attribute '__dict__' of 'cfg' objects>, '__weakref__': <attribute '__weakref__' of 'cfg' objects>, '__doc__': None}


In [ ]:
ROOT = Path.cwd().parent
hms_path=ROOT/"content/"
data_path=ROOT/"content/train_eegs/"
processed_data_path=ROOT/"preprocessed_data/"
model_path = "/content/drive/MyDrive/HMS/model/"

In [ ]:
train = pl.read_csv(ROOT/"/content/drive/MyDrive/HMS/data/train.csv")
TARGET=train[:,-6:].columns
train=train.with_columns(train[TARGET].sum(axis=1).alias("SUM"))

train=train.with_columns(pl.when(pl.col("SUM")>9).
                   then(0).otherwise(1).alias("T_STAGE")
                    ).drop("SUM")


train[TARGET]=train[TARGET]/train[TARGET].sum(axis=1)


<ipython-input-8-3bddfbdb8e80>:3: DeprecationWarning: The `axis` parameter for `DataFrame.sum` is deprecated. Use `DataFrame.sum_horizontal()` to perform horizontal aggregation.
  train=train.with_columns(train[TARGET].sum(axis=1).alias("SUM"))
<ipython-input-8-3bddfbdb8e80>:10: DeprecationWarning: The `axis` parameter for `DataFrame.sum` is deprecated. Use `DataFrame.sum_horizontal()` to perform horizontal aggregation.
  train[TARGET]=train[TARGET]/train[TARGET].sum(axis=1)


In [ ]:
def set_random_seed(seed: int = 42, deterministic: bool = True):
    """Set seeds"""
    random.seed(seed)
    np.random.seed(seed)
    os.environ["PYTHONHASHSEED"] = str(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)  # type: ignore
    torch.backends.cudnn.deterministic = deterministic

In [ ]:
sgkf = StratifiedGroupKFold(n_splits=cfg.folds, shuffle=True, random_state=cfg.seed)
sp_dat = train.filter(pl.col("eeg_sub_id")==0)
train  = train.with_columns(pl.lit(-1).alias("CV_NO"))

for i, (train_idx, val_idx) in enumerate(sgkf.split(sp_dat, y=sp_dat["expert_consensus"], groups=sp_dat["patient_id"])):
    p_id = sp_dat[val_idx,"patient_id"].unique()
    train=train.with_columns(
        pl.when(pl.col("patient_id").is_in(p_id)).
        then(i).
        otherwise(pl.col("CV_NO")).alias("CV_NO")
    )



In [ ]:
def run_train_valid(i,SUB_NO,train,HMS_EEG_Dataset,cfg,model,val_result,T_STAGE):
    criterion   = KLDivLossWithLogits().to(cfg.device)
    loss_func_val = KLDivLossWithLogitsForVal()
    optimizer   = torch.optim.AdamW(params=model.parameters(), lr=cfg.lr, weight_decay=cfg.weight_decay)
    lr_scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=cfg.epoch, eta_min=0.0001)
    scaler = amp.GradScaler(enabled=True)

    t_d  = train.filter(pl.col("CV_NO") != i).filter(pl.col("eeg_sub_id") == SUB_NO).to_pandas()
    v_d  = train.filter(pl.col("CV_NO") == i).filter(pl.col("eeg_sub_id") == SUB_NO).to_pandas()

    train_dataset = HMS_EEG_Dataset(label_data=t_d,duration=cfg.duration)
    valid_dataset = HMS_EEG_Dataset(label_data=v_d,duration=cfg.duration,valid_mode=True)

    train_dataloader = DataLoader(train_dataset, batch_size=cfg.batch_size,shuffle=True)
    valid_dataloader = DataLoader(valid_dataset, batch_size=cfg.batch_size,shuffle=False)


    val_all= train.filter(pl.col("CV_NO")==i)
    valid_dataset_all = HMS_EEG_Dataset(label_data=val_all.to_pandas(),duration=cfg.duration,valid_mode=True)
    valid_dataloader_all = DataLoader(valid_dataset_all, batch_size=cfg.batch_size,shuffle=False)

    val_loss= 1.0e+09
    best_val_loss = 1.0e+08
    erstp=0
    for epoch in range(cfg.epoch):

        train_loss = train_one_epoch(train_dataloader,epoch,cfg,model,criterion,
                                     optimizer,scaler,lr_scheduler,val_loss,i)

        val_loss,_ = valid_one_epoch(valid_dataloader,epoch,cfg,model,loss_func_val)

        val_loss2,val_pred = valid_one_epoch(valid_dataloader_all,epoch,cfg,model,loss_func_val)

        if val_loss2 < best_val_loss:
            best_epoch = epoch
            best_val_loss = val_loss2
            torch.save(model.state_dict(), "/content/drive/MyDrive/HMS/model/"+ f'HMS_model_EX{cfg.EXP_NO}_sub{SUB_NO}_cv_{i}.pth')
            val_pred.to_csv("/content/drive/MyDrive/HMS/val_pred/"+f"val_pred_EX{cfg.EXP_NO}_sub{SUB_NO}_cv_{i}.csv")
            erstp=0
        else:
            erstp+=1
            if erstp > cfg.early_stop:
                break

        print(f"cv_{i}",f" epoch{epoch}"," train_loss:",train_loss," val_loss:",val_loss," val_loss2:",val_loss2,"best_val_loss:",best_val_loss)
        tmp = pl.DataFrame({"cv":i,"epoch":epoch,"train_loss":train_loss,"val_loss_sub0":val_loss,"val_loss_all":val_loss2,"best_val_loss:":best_val_loss})
        val_result = pl.concat([val_result,tmp])
        val_result.write_csv(f"/content/drive/MyDrive/HMS/val_result/EXP{cfg.EXP_NO}_sub{SUB_NO}.csv")

    return val_result

In [ ]:

val_result = pl.DataFrame()
for CV_NO in range(5):
    set_random_seed(cfg.seed)

    M_PATH = model_path + f'HMS_model_EX{cfg.EXP_NO}_sub{SUB_NO}_cv_{CV_NO}.pth'

    extractor = CNNSpectrogram(in_channels=cfg.in_channels,kernel_sizes=cfg.kernel_sizes)
    #extractor = WavenetFeatureExtractor()
    model     = HMSHBACSpecModel(model_name=cfg.model_name, pretrained=True, num_classes=6, in_channels=len(cfg.kernel_sizes),feature_extractor=extractor).to(cfg.device)
    #model = wavenet_classifier().to(cfg.device)
    T_STAGE = None
    val_result = run_train_valid(CV_NO,SUB_NO,train,
                                 HMS_EEG_Dataset,cfg,model,val_result,T_STAGE)

    #extractor = CNNSpectrogram(kernel_sizes=cfg.kernel_sizes)
    #model     = HMSHBACSpecModel(model_name=cfg.model_name, pretrained=False, num_classes=6, in_channels=len(cfg.kernel_sizes),feature_extractor=extractor).to(cfg.device)
    #model.load_state_dict(torch.load(M_PATH))
    #T_STAGE = 1
    #val_result = run_train_valid(CV_NO,SUB_NO,train,HMS_EEG_Dataset,cfg,model,val_result,T_STAGE)











/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Epoch 0/20:   0%|          | 0/283 [00:00<?, ?it/s]

cv_0  epoch0  train_loss: 0.897981097125333  val_loss: 0.971813976764679  val_loss2: 1.0677461624145508 best_val_loss: 1.0677461624145508


Epoch 1/20:   0%|          | 0/283 [00:00<?, ?it/s]

cv_0  epoch1  train_loss: 0.7251358465044746  val_loss: 0.839165449142456  val_loss2: 0.943149983882904 best_val_loss: 0.943149983882904


...

Epoch 0/20:   0%|          | 0/261 [00:00<?, ?it/s]

cv_3  epoch0  train_loss: 0.9119799418010932  val_loss: 0.9955409169197083  val_loss2: 1.0301836729049683 best_val_loss: 1.0301836729049683


Epoch 1/20:   0%|          | 0/261 [00:00<?, ?it/s]

cv_3  epoch1  train_loss: 0.735409358680477  val_loss: 0.9248588681221008  val_loss2: 0.8739380836486816 best_val_loss: 0.8739380836486816


Epoch 2/20:   0%|          | 0/261 [00:00<?, ?it/s]